In [2]:
# Imports 
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline 

In [3]:
# That Data
df = pd.read_json('../Data/data.json')
df.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,premium,1297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA


In [4]:
# Make fraud column 
df['fraud'] = df.acct_type.apply(lambda x: 1 if 'fraud' in x else 0)

In [5]:
df.fraud.value_counts()

0    13044
1     1293
Name: fraud, dtype: int64

In [ ]:
# Get columns 
columns = list(df.columns)

# See What they looks like 
df.info()

In [ ]:
# Desccribe Int variables 
df.describe().T

In [ ]:
# Get missing varaiabels 
missing_values = []
for col in columns:
    if df[col].isna().any():
        missing_values.append(col)
        
print(f'Number: {len(missing_values)} \n\nVariables with missing values:\n{missing_values}')

In [ ]:
# get variabels by type 
var_floats = []
var_ints = []
var_objects = []

for col in columns:
    if df[col].dtype == 'int64':
        var_ints.append(col)
    elif df[col].dtype == 'float64':
        var_floats.append(col)
    else:
        var_objects.append(col)

print(f' Floats: {var_floats} \n\n Ints: {var_ints} \n\n Objects: {var_objects}')

# Notes 

**Isolation Forest**

In [ ]:
# Show Int data distribution 
def plot(data, vars_, r, c):
    fig, axs = plt.subplots(r, c, figsize = (10,10))

    for i, ax in enumerate(axs.flatten()):
        ax.hist(data[vars_[i]])
        ax.set_xlabel(vars_[i])
        ax.set_ylabel('freq')

    plt.tight_layout()
plot(df, var_ints, 6, 3)

In [ ]:
df.user_type.value_counts()

In [ ]:
# Show float data distribution 
plot(df, var_floats, 3, 3)

In [ ]:
# Peek Object Variables 
for v in var_objects:
    print(f'    {v} \n{df[v].value_counts()} \n\n\n')

In [ ]:
# Return all events with no payouts 
nopayouts_mask = df.previous_payouts.apply(lambda x: True if len(x) <= 0 else False)
df[nopayouts_mask]

In [ ]:
# No pay out that are farud
df2 = df[~nopayouts_mask]

In [ ]:
df.fraud.sum()

In [ ]:
df2_fraud = df2[df2.fraud == 1]

In [ ]:
df2_fraud

In [ ]:
plot(df2_fraud, var_ints, 6, 3)

In [ ]:
plot(df2_fraud, var_floats, 3, 3)

In [ ]:
# Peek Object Variables 
for v in var_objects:
    print(f'    {v} \n{df2_fraud[v].value_counts()} \n\n\n')
    
# delivery_method     991 non-null    float64
# has_header          339 non-null    float64
# 42  venue_name          203 non-null    object
# 25  org_facebook        289 non-null    float64
# 31  sale_duration       288 non-null    float64

In [ ]:
# Peek Object Variables 
for v in ['payee_name', 'org_name']:
    print(f'    {v} \n{df[v].value_counts()} \n\n\n')

In [ ]:
df.fraud.sum()

In [ ]:
# Check to see if payee is a real person 
df['missing_payee'] = df.payee_name.apply(lambda x: 1 if len(x) < 3 else 0)

## ------------ Fraud Payee and Pay to

In [ ]:
# Missing payee
mask_none_missing_payee = df['missing_payee'] == 0

In [ ]:
# None Missing payee that are fraud 
temp = df[(df['missing_payee'] == 0) & (df['fraud'] == 1)]

In [ ]:
# get all the people who did not pay out to some one but was fraud 
mask = temp.previous_payouts.apply(lambda x: True if len(x) <= 0 else False)

In [ ]:
temp.loc[~mask, ['payee_name']]

In [ ]:
temp[~mask].previous_payouts[11504]

# ---- Important fraud factors 
1. No previous pay outs 
2. Pay out is to no one 
3. Payout is to payee



mask = df.previous_payouts.apply(lambda x: True if len(x) <= 0 else False)
df[mask]

In [6]:
def not_premium(x):
    return 1 if x.lower() != 'premium' else 0

def no_previous_payout(x):
    return 1 if len(x) <= 0 else 0

def payout_name_flag(x):
    for d in x:
        if len(d['name']) > 3:
            return 1
    return 0

def payout_toself(payee, payouts):
    for pay in payouts:
        if payee.lower() in pay['name'].lower():
            return 1
    return 0

In [8]:
temp = dict()
temp = {'target': df['fraud'].copy(), 
        'premium': df.acct_type.apply(lambda x: not_premium(x)),
        'previous_payout': df.previous_payouts.apply(lambda x: no_previous_payout(x)),
        'no_payout_name': df.previous_payouts.apply(lambda x: payout_name_flag(x)),
        'payout_toself': df.apply(lambda x: payout_toself(x.payee_name, x.previous_payouts), axis = 1)
       }

In [17]:
df.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state',
       'fraud'],
      dtype='object')

In [23]:
df[df.org_name.duplicated()]

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,fraud
8,premium,1298185200,4388,4,US,USD,0.0,"<p style=""text-align: center;""><font face=""boo...",credosf.com,1294428122,...,0,1294428121,3,465 California Street,US,37.792847,-122.402082,,,0
9,premium,1297314000,974,12,US,USD,0.0,<p><em>WHEN JEKYLL MET HYDE</em></p>\r\n<p>The...,themagnetictheatre.org,1294428286,...,71,1288276103,3,372 Depot Street,US,35.580468,-82.563855,The Magnetic Field,NC,0
16,premium,1300075200,1595,6,US,USD,0.0,"<table style=""width: 100%;"" border=""0"">\n<tbod...",racetonowhere.com,1296775079,...,371,1264709634,3,376 Hale Street,US,42.553499,-70.845015,"Endicott College, Center for the Arts, Rose Pe...",MA,0
18,premium,1313881200,3387,11,US,USD,1.0,"<h3 style=""text-align: center;"">Rob Holmes is ...",ipcybercrime.com,1297481644,...,709,1236178830,4,10720 Cambie Rd.,CA,49.184581,-123.106135,,,0
26,fraudster,1299607200,870,11,US,USD,0.0,"<h1 class=""post"">Welcome</h1>\r\n<p> </p>\r\n<...",hotmail.com,1299171416,...,42,1295576777,3,400 Fifth Avenue,US,40.750228,-73.983714,The Setai,NY,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14328,premium,1369159200,8149,5,GB,GBP,0.0,"<p><strong style=""font-size: medium;"">What you...",chatterboxsocialmedia.co.uk,1359809733,...,749,1295117568,4,Clavering House,GB,54.968276,-1.612204,,,0
14329,premium,1372010400,11930,8,GB,GBP,0.0,"<p><strong style=""font-size: medium;"">What you...",chatterboxsocialmedia.co.uk,1359809804,...,749,1295117568,4,Clavering House,GB,54.968276,-1.612204,,,0
14331,premium,1370736000,4560,5,US,USD,1.0,"<p class=""MsoNormal"" style=""margin: 0in 0in 0....",revolutiongroup.com,1364417805,...,111,1354810526,3,400 North High Street,US,39.953948,-82.998998,Greater Columbus Convention Center,OH,0
14332,fraudster_event,1361689200,667,0,US,USD,0.0,<p>Celebrates this wonderful holiday with the ...,yahoo.com,1360297993,...,0,1360297993,1,6100 North Charles Street,US,39.373780,-76.629921,Elkridge Club,MD,1
